# Important Libraries

In [58]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### DVCS (Unpolarized) Amplitude Squared Contributions

The DVCS amplitude squared can be written as 

$$ |\mathcal{M}_{DVCS}|^{2} = \frac{e^{6}}{y^{2} Q^{2}} \left[ c_{0, DVCS} + c_{1, DVCS} cos(\phi) + s_{1, DVCS} sin(\phi) + c_{2, DVCS} cos(2 \phi) + s_{2, DVCS} sin(2 \phi)  \right].$$

So, we need to code five coefficients, each are a crazy function of ten thousand other things. Let's see what we need.

# Important Constants:

In [80]:
_ELECTROMAGNETIC_FINE_STRUCTURE_CONSTANT = (1. / 137.0359998)
_ELECTROMAGNETIC_FINE_STRUCTURE_CONSTANT_INVERSE = 0.0072973525693
_PI_DECIMAL = 3.141592653589793238462643383279502884197
_PI_RADIANS = _PI_DECIMAL / 180.
_MASS_OF_PROTON_IN_GEV = .93827208816
_MASS_OF_PROTON_SQUARED_IN_GEV_SQUARED = _MASS_OF_PROTON_IN_GEV *_MASS_OF_PROTON_IN_GEV
_CONVERT_GEV_TO_NANOBARNS_FACTOR = .389379 * 1000000.
_EXPERIMENTALLY_DETERMINED_CONSTANT_IN_ELECTRIC_FORM_FACTOR = 0.710649
_PROTON_MAGNETIC_MOMENT = 2.79284734463

## $\varepsilon = \frac{2 x_{B} m_{p}}{Q}$:

In [60]:
def calculate_kinematics_epsilon(
        squared_Q_momentum_transfer: float, 
        x_Bjorken: float, 
        verbose = True) -> float:
    """
    Description
    --------------
    Calculate epsilon, which is just a ratio of kinematic quantities:
    \epsilon := 2 * m_{p} * x_{B} / Q

    Parameters
    --------------
    squared_Q_momentum_transfer: (float)
        kinematic momentum transfer to the hadron. 

    x_Bjorken: (float)
        kinematic Bjorken X

    verbose: (bool)
        Debugging console output.
    

    Notes
    --------------
    """
    if verbose:
        print(f"> Received mp (mass of proton): {_MASS_OF_PROTON_IN_GEV}")
        print(f"> Received xB (Bjorken x): {x_Bjorken}")
        print(f"> Received Q^2  (squared momentum transfer): {squared_Q_momentum_transfer}")
        print(f"> Calculated Q with NumPy as: {np.sqrt(squared_Q_momentum_transfer)}")
            
    try:

        # (1): Calculate Epsilon right away:
        epsilon = (2. * x_Bjorken * _MASS_OF_PROTON_IN_GEV) / np.sqrt(squared_Q_momentum_transfer)

        if verbose:
            print(f"> Calculated epsilon to be: {epsilon}")

        # (2): Return Epsilon:
        return epsilon
    
    except Exception as E:
        print(f"> Error in computing kinematic epsilon:\n> {E}")
        return None

## $\varepsilon^{2} = \frac{4 x_{B}^{2} m_{p}^{2}}{Q^{2}}$:

In [61]:
def calculate_kinematics_epsilon_squared(
        squared_Q_momentum_transfer: float, 
        x_Bjorken: float, 
        verbose = True) -> float:
    """
    Description
    --------------
    Calculate epsilon, which is just a ratio of kinematic quantities:
    \epsilon^{2} := 4 * m_{p}^{2} * x_{B}^{2} / Q^{2}

    Parameters
    --------------
    squared_Q_momentum_transfer: (float)
        kinematic momentum transfer to the hadron. REALLY IMPORTANT
        THAT IT'S THE SQUARE OR MOMENTUM TRANSFER.

    x_Bjorken: (float)
        kinematic Bjorken X

    verbose: (bool)
        Debugging console output.
    

    Notes
    --------------
    """
    try:
        epsilon_squared = (4. * _MASS_OF_PROTON_IN_GEV**2 * x_Bjorken**2) / squared_Q_momentum_transfer

        if verbose:
            print(f"> Received mp (mass of proton): {_MASS_OF_PROTON_IN_GEV}")
            print(f"> Received xB (Bjorken x): {x_Bjorken}")
            print(f"> Received Q  (momentum transfer): {squared_Q_momentum_transfer}")
            print(f"> Calculated epsilon squared to be: {epsilon_squared}")

        return epsilon_squared
    
    except Exception as E:
        print(f"> Error in computing kinematic epsilon:\n> {E}")
        return None

## y = $\frac{p_{1} \cdot q_{1}}{p_{1} \cdot k} = \frac{Q}{\epsilon k}$

In [62]:
def calculate_kinematics_lepton_energy_fraction_y(
        epsilon: float, 
        squared_Q_momentum_transfer: float, 
        lab_kinematics_k: float, 
        verbose = True) -> float:
    """
    Description
    --------------
    Calculate y, which measures the leptopn energy fraction.
    y^{2} := \frac{ \sqrt{Q^{2}} }{ \sqrt{\epsilon^{2}} k }

    Parameters
    --------------
    epsilon: (float)
        derived kinematics

    squared_Q_momentum_transfer: (float)
        Q^{2} momentum transfer to the hadron

    lab_kinematics_k: (float)
        lepton momentum loss

    verbose: (bool)
        Debugging console output.

    Notes
    --------------

    """
    try:
        lepton_energy_fraction_y = np.sqrt(squared_Q_momentum_transfer) / (epsilon * lab_kinematics_k)

        if verbose:
            print(f"> Calculated y to be: {lepton_energy_fraction_y}")

        return lepton_energy_fraction_y
    
    except Exception as E:
        print(f"> Error in computing lepton_energy_fraction_y:\n> {E}")
        return None

## $\xi = x_{B} (\frac{1 + \frac{t}{2 Q^{2}}}{2 - x_{B} + \frac{x_{B} t }{Q^{2}}})$

In [63]:
def calculate_kinematics_skewness_parameter(
        squared_Q_momentum_transfer: float, 
        x_Bjorken: float, 
        hadronic_four_momentum_change_t_delta_squared: float, 
        verbose = False) -> float:
    """
    Description
    --------------
    Calculate the Skewness Parameter
    x_{i} = x_{B} * (1 + \frac{ t Q^{2} }{ 2 } ) ... FUCK YOU

    Parameters
    --------------
    squared_Q_momentum_transfer: (float)
        kinematic momentum transfer to the hadron

    x_Bjorken: (float)
        kinematic Bjorken X

    verbose: (bool)
        Debugging console output.
    

    Notes
    --------------
    """
    try:

        # (1): The Numerator:
        numerator = (1. + (hadronic_four_momentum_change_t_delta_squared / (2. * squared_Q_momentum_transfer)))

        # (2): The Denominator:
        denominator = (2. - x_Bjorken + (x_Bjorken * hadronic_four_momentum_change_t_delta_squared / squared_Q_momentum_transfer))

        # (3): Calculate the Skewness Parameter:
        skewness_parameter = x_Bjorken * numerator / denominator

        if verbose:
            print(f"> Received Q (momentum transfer): {squared_Q_momentum_transfer}")
            print(f"> Received x_B (Bjorken x) {x_Bjorken}")
            print(f"> Received t (hadronic momentum change): {hadronic_four_momentum_change_t_delta_squared}")
            print(f"> Calculated generalized Bjorken to be: {skewness_parameter}")

        return skewness_parameter
    
    except Exception as E:
        print(f"> Error in computing kinematic generalized Bjorken x:\n> {E}")
        return None

## $t_{0} = - \frac{4 \xi^{2} m_{p}^{2}}{1 - \xi^{2}}$

In [64]:
def calculate_kinematics_minimum_t(
        skewness_parameter: float, 
        verbose = False) -> float:
    """
    Description
    --------------
    Calculate the minimum value of t:
    (Expression is causing issues in cell evaluation. Will fill in later.)

    Parameters
    --------------
    skewness_parameter: (float)
        skewness parameter

    Returns
    --------------
    t_minimum: (float)
        minimum t

    Notes
    --------------
    """
    try:

        # (1): Calculate the Numerator:
        numerator = 4. * skewness_parameter**2 * _MASS_OF_PROTON_IN_GEV**2

        # (2): Calculate the Denominator:
        denominator = 1 - (skewness_parameter**2)

        # (3): Obtain the t minimum
        t_minimum = (-1.) * numerator / denominator

        if verbose:
            print(f"> Received mass of proton: {_MASS_OF_PROTON_IN_GEV}")
            print(f"> Received xi (skewness): {skewness_parameter}")
            print(f"> Calculated the minimum value of t to be: {t_minimum}")

        return t_minimum

    except Exception as E:
        print(f"> Error calculating minimum t:\n> {E}")
        return None

## $K^{2}$

In [65]:
def calculate_kinematics_k_squared(
        epsilon: float, 
        squared_Q_momentum_transfer: float, 
        x_Bjorken: float, 
        lepton_energy_fraction_y: float,
        squared_hadronic_momentum_transfer_t: float,
        squared_hadronic_momentum_transfer_t_minimum: float,
        verbose = True) -> float:
    """
    Description
    --------------
    Equation (30) in the BKM Formalism, available
    at this link: https://arxiv.org/pdf/hep-ph/0112108.pdf

    Parameters
    --------------
    epsilon: (float)

    squared_Q_momentum_transfer: (float)

    x_Bjorken: (float)

    lepton_energy_fraction_y: (float)

    squared_hadronic_momentum_transfer_t: (float)

    squared_hadronic_momentum_transfer_t_minimum: (float)

    verbose: (bool)
        Debugging console output.

    Returns
    --------------
    k_squared : (float)
        result of the operation
    
    Notes
    --------------
    (1): k-dot-delta shows up in computing the lepton
        propagators. We need to get K^{2} first. It is Eq. (29) 
        in the following paper: https://arxiv.org/pdf/hep-ph/0112108.pdf
    """
    
    try:

        # (1): Calculate recurring quantity 1 - xB -- this IS multiplicative quantity one!
        one_minus_x_bjorken = 1 - x_Bjorken

        # (2): Multiplicative Quantity Two:
        second_multiplicative_quantity = 1. - lepton_energy_fraction_y - (lepton_energy_fraction_y**2 * epsilon**2 / 4.)

        # (3): Multiplicative Quantity Three:
        third_multiplicative_quantity = 1. - (squared_hadronic_momentum_transfer_t_minimum / squared_hadronic_momentum_transfer_t)

        # (3): Multiplicative Quantity Four - First Term:
        fourth_multiplicative_quantity_first_term = np.sqrt(1 + epsilon**2)

        # (4): Multiplicative Quantity Four - Second Term - Numerator:
        fourth_multiplicative_quantity_second_term_numerator = 4. * x_Bjorken * one_minus_x_bjorken + epsilon**2

        # (5): Multiplicative Quantity Four - Second Term - Denominator:
        fourth_multiplicative_quantity_second_term_denominator = 4. * one_minus_x_bjorken

        # (6): Multiplicative Quantity Four - Prefactor:
        fourth_multiplicative_quantity_second_term_prefactor = ((squared_hadronic_momentum_transfer_t - squared_hadronic_momentum_transfer_t_minimum ) / squared_Q_momentum_transfer)

        # (7): Multiplicative Quantity Four:
        fourth_multiplicative_quantity_second_term = fourth_multiplicative_quantity_second_term_prefactor* fourth_multiplicative_quantity_second_term_numerator / fourth_multiplicative_quantity_second_term_denominator

        # (8): Multiplicative Quantity Four in Whole:
        fourth_multiplicative_quantity = fourth_multiplicative_quantity_first_term + fourth_multiplicative_quantity_second_term

        # (9): Calculate the prefactor of the entire thing:
        k_squared_prefactor = squared_hadronic_momentum_transfer_t / squared_Q_momentum_transfer
    
        # (9): Calculate the final quantity: K^{2}
        k_squared = -1. * k_squared_prefactor * one_minus_x_bjorken * second_multiplicative_quantity * third_multiplicative_quantity * fourth_multiplicative_quantity

        if verbose:
            print(f"> Calculated k_squared to be: {k_squared}")

        # (10) Return K^{2}:
        return k_squared

    except Exception as E:
        print(f"> Error in calculating K^2\n> {E}")
        return None

#### $\mathcal{C}_{unp}^{DVCS}(\mathcal{F}, \mathcal{F}^{*})$

$$\mathcal{C}_{unp}^{DVCS}(\mathcal{F}, \mathcal{F}^{*}) =  ...$$


$$ \frac{Q^{2} (Q^{2} + x_{B} t)}{((2 - x_{B}) Q^{2} + x_{B} t)^{2}} \left[ 4 ( 1 - x_{B}) \mathcal{H} \mathcal{H}^{*} + 4 \left( 1 - x_{B} + \frac{2 Q^{2} + t}{Q^{2} + x_{B} t} \frac{\varepsilon^{2}}{4} \right) \tilde{\mathcal{H}} \tilde{\mathcal{H}^{*}} \right] $$

With the assignments:

$$
\begin{align*}

    a & := \frac{t}{4 m_{p}^{2}} \\
    b & := 1 - x_{B} \\
    c & := Q^{2} + x_{B} t \\
    d & := Q^{2} (Q^{2} + x_{B} t) = Q^{2} c \\
    e & := (2 - x_{B} ) Q^{2} + x_{B} t \\
    f & := Q^{2} + t \\

\end{align*} $$

we can write the insane coefficient as:

$$
    \mathcal{C}_{unp}^{DVCS}(\mathcal{F}, \mathcal{F}^{*}) = \frac{d}{e^{2}} \left[ 4 b \mathcal{H}\mathcal{H}^{*} + 4 \left( b + \frac{2 Q^{2} + t}{c} \frac{\varepsilon^{2}}{4} \right) \tilde{\mathcal{H} \tilde{\mathcal{H}^{*}}} - \frac{x_{B} f^{2}}{d} \left( \mathcal{H} \mathcal{E}^{*} + \mathcal{E} \mathcal{H}^{*} \right) - \frac{x_{B} Q^{2}}{c} \left( \tilde{\mathcal{H}} \tilde{\mathcal{E}^{*}} + \tilde{\mathcal{E}} \tilde{\mathcal{H}^{*}} \right) - \left( \frac{x_{B} f^{2}}{d} + \frac{e^{2}}{d} a \right) \mathcal{E} \mathcal{E}^{*} - \frac{x_{B} Q^{2}}{c} a \tilde{\mathcal{E}} \tilde{\mathcal{E}^{*}} \right]
$$

In [73]:
def calculate_C_function_of_CFFs_unpolarized(
        epsilon: float,
        squared_Q_momentum_transfer: float,
        x_Bjorken: float,
        squared_hadronic_momentum_transfer_t: float,
        compton_form_factor_h: float,
        compton_form_factor_h_tilde: float,
        compton_form_factor_e: float,
        compton_form_factor_e_tilde: float,
        verbose = True
    ) -> float:
    """
    Description
    --------------
    This is not a coefficient in the Fourier expansion of the DVCS amplitude squared,
    but rather is just part of that coefficient. Specifically, it is a massive function
    of kinematic invariants and the Compton Form Factors (CFFs).

    WARNING:
    The BKM02 Formalism includes this as Equation (66) in:
    https://arxiv.org/pdf/hep-ph/0112108.pdf

    The BKM10 Formalism includes this as Equation (2.22) in:
    https://arxiv.org/pdf/1005.5209.pdf

    Parameters
    --------------
    epsilon: (float)
    
    squared_Q_momentum_transfer: (float)

    x_Bjorken: (float)

    squared_hadronic_momentum_transfer_t: (float)

    compton_form_factor_h: (float)

    compton_form_factor_h_tilde: (float)

    compton_form_factor_e: (float)

    compton_form_factor_e_tilde: (float)

    verbose: (bool)
        Debugging console output.

    Returns
    --------------
    c_dvcs_unpolarized_ff: (float)

    Notes
    --------------
    """

    if verbose:
        print(f"> [{__name__}]: Now beginning calculation of C_function_unp.")

    try:

        # (1): Calculate the "a" factor:
        a_factor = squared_hadronic_momentum_transfer_t / (4. * _MASS_OF_PROTON_IN_GEV**2)
        
        if verbose:
            print(f"> [{__name__}]: Calculated `a_factor` to be: {a_factor}")

        # (2): Calculate the "b" factor:
        b_factor = 1. - x_Bjorken

        if verbose:
            print(f"> [{__name__}]: Calculated `b_factor` to be: {b_factor}")

        # (3): Calculate the "c" factor:
        c_factor = squared_Q_momentum_transfer + x_Bjorken * squared_hadronic_momentum_transfer_t

        if verbose:
            print(f"> [{__name__}]: Calculated `c_factor` to be: {c_factor}")

        # (4): Calculate the "d" factor:
        d_factor = squared_hadronic_momentum_transfer_t * c_factor

        if verbose:
            print(f"> [{__name__}]: Calculated `d_factor` to be: {d_factor}")

        # (5): Calculate the "e" factor:
        e_factor = (2. - x_Bjorken) * squared_Q_momentum_transfer + x_Bjorken * squared_hadronic_momentum_transfer_t

        if verbose:
            print(f"> [{__name__}]: Calculated `e_factor` to be: {e_factor}")

        # (6): Calculate the "f" factor:
        f_factor = squared_Q_momentum_transfer + squared_hadronic_momentum_transfer_t

        if verbose:
            print(f"> [{__name__}]: Calculated `f_factor` to be: {f_factor}")

        # (7): Calculate the prefactor:
        prefactor = d_factor / e_factor**2

        if verbose:
            print(f"> [{__name__}]: Calculated `prefactor` to be: {prefactor}")

        # (8): Calculate the scaling on the H and H-tilde CFFs:
        cff_h_h_tilde_contribution_first_term = 4. * b_factor * compton_form_factor_h * compton_form_factor_h

        if verbose:
            print(f"> [{__name__}]: Calculated `cff_h_h_tilde_contribution_first_term` to be: {cff_h_h_tilde_contribution_first_term}")


        # (9): Calculate the scaling on the H and H-tilde CFFs:
        cff_h_h_tilde_contribution_second_term = 4. * compton_form_factor_h_tilde * compton_form_factor_h_tilde * (b_factor + ((2. * squared_Q_momentum_transfer + squared_hadronic_momentum_transfer_t) * epsilon) / (4. * c_factor)) 

        if verbose:
            print(f"> [{__name__}]: Calculated `cff_h_h_tilde_contribution_second_term` to be: {cff_h_h_tilde_contribution_second_term}")
        
        # (10): Calculate the scaling on mixing H and E CFFs (with relative complex-conjugation):
        cff_e_h_mixing_contribution = -1. * x_Bjorken * f_factor**2 * (compton_form_factor_h * compton_form_factor_e + compton_form_factor_h * compton_form_factor_e) / d_factor

        if verbose:
            print(f"> [{__name__}]: Calculated `cff_e_h_mixing_contribution` to be: {cff_e_h_mixing_contribution}")
        
        # (11): Calculate the scaling on mixing H-Tilde and E-Tilde CFFs:
        cff_e_tilde_h_tilde_mixing_contribution = -1. * x_Bjorken * squared_Q_momentum_transfer * (compton_form_factor_h_tilde * compton_form_factor_e_tilde + compton_form_factor_h_tilde * compton_form_factor_e_tilde) / c_factor

        if verbose:
            print(f"> [{__name__}]: Calculated `cff_e_tilde_h_tilde_mixing_contribution` to be: {cff_e_tilde_h_tilde_mixing_contribution}")
        
        # (12): Calculate the scaling on E/EStar CFFs:
        cff_e_estar_contribution = -1. * (x_Bjorken * f_factor**2 / d_factor + e_factor**2 * a_factor / d_factor) * compton_form_factor_e * compton_form_factor_e

        if verbose:
            print(f"> [{__name__}]: Calculated `cff_e_estar_contribution` to be: {cff_e_estar_contribution}")
        
        # (13): Calculate the scaling on E-Tilde/E-TildeStar CFFs:
        cff_e_tilde_e_tilde_star_contribution = -1. * x_Bjorken * squared_Q_momentum_transfer * a_factor * c_factor * compton_form_factor_e_tilde * compton_form_factor_e_tilde

        if verbose:
            print(f"> [{__name__}]: Calculated `cff_e_tilde_e_tilde_star_contribution` to be: {cff_e_tilde_e_tilde_star_contribution}")
        
        # (14): Put together everything
        C_function_of_CFFs_unpolarized = prefactor * (cff_h_h_tilde_contribution_first_term + cff_h_h_tilde_contribution_second_term + cff_e_h_mixing_contribution + cff_e_tilde_h_tilde_mixing_contribution + cff_e_estar_contribution + cff_e_tilde_e_tilde_star_contribution)

        if verbose:
            print(f"> [{__name__}]: Calculated `C_function_of_CFFs_unpolarized` to be: {C_function_of_CFFs_unpolarized}")

        # (15): Return the huge number:
        return C_function_of_CFFs_unpolarized

    except Exception as E:
        print(f"> Error computing C_function_of_CFFs_unpolarized:\n> {E}")
        return None

## $\mathcal{C}_{T,unp}^{DVCS}(\mathcal{F}_{T}, \mathcal{F}^{*})$

In [74]:
def calculate_dvcs_coefficient_T_unpolarized_form_factors(
        x_Bjorken: float,
        squared_hadronic_momentum_transfer_t: float,
        compton_form_factor_h: float,
        compton_form_factor_h_tilde: float,
        compton_form_factor_hT: float,
        compton_form_factor_hT_tilde: float,
        compton_form_factor_e: float,
        compton_form_factor_e_tilde: float,
        compton_form_factor_eT: float,
        compton_form_factor_eT_tilde: float,
        verbose = True
    ) -> float:
    """
    Description
    --------------
    We calculate the DVCS coefficient for the unpolarized
    target that is a function of all the damn form factors. This
    coefficient differs from the first one because it involves
    "gluon transversity," whatever that is.

    Parameters
    --------------
    x_Bjorken: (float)

    squared_hadronic_momentum_transfer_t: (float)

    compton_form_factor_h: (float)

    compton_form_factor_h_tilde: (float)

    compton_form_factor_hT: (float)

    compton_form_factor_hT_tilde: (float)

    compton_form_factor_e: (float)

    compton_form_factor_e_tilde: (float)

    compton_form_factor_eT: (float)

    compton_form_factor_eT_tilde: (float)

    verbose: (bool)
        Debugging console output.

    Returns
    --------------
    c_dvcs_T_unpolarized_ff: (float)

    Notes
    --------------
    (1): The equation for this coefficient comes from
        Eq. (76) in this paper:
        https://arxiv.org/pdf/hep-ph/0112108.pdf
    """
    
    if verbose:
        print(f"> [{__name__}]: Now beginning calculation of C_function_unp with F stars.")

    try:

        # (1): Compute the recurring term 2 - xB
        two_minus_xB = 2. - x_Bjorken

        # (2): Compute the first term in the brackets:
        first_bracket_term = compton_form_factor_hT * (two_minus_xB * compton_form_factor_e - x_Bjorken * compton_form_factor_e_tilde )

        # (3): Compute the second term in the brackets:
        second_bracket_term = -2. * two_minus_xB * compton_form_factor_hT_tilde * compton_form_factor_hT_tilde * (compton_form_factor_h + (squared_hadronic_momentum_transfer_t * compton_form_factor_e_tilde / (4. * _MASS_OF_PROTON_IN_GEV**2)))

        # (4): Compute the third term in the brackets:
        third_bracket_term = -1. * compton_form_factor_eT * (two_minus_xB * compton_form_factor_h - x_Bjorken * compton_form_factor_h_tilde)

        # (5): Compute the fourth term in the brackets:
        fourth_bracket_term = compton_form_factor_eT_tilde * (x_Bjorken * compton_form_factor_h + x_Bjorken * compton_form_factor_e) - two_minus_xB * compton_form_factor_h_tilde

        # (6): Compute the entire coefficient:
        c_dvcs_T_unpolarized_ff = (first_bracket_term + second_bracket_term + third_bracket_term + fourth_bracket_term) / two_minus_xB**2

        if verbose:
            print(f"> Computed c_dvcs_T_unpolarized_ff to be: {c_dvcs_T_unpolarized_ff}")

        return c_dvcs_T_unpolarized_ff


    except Exception as E:
        print(f"> Error computing c_dvcs_T_unpolarized_ff:\n> {E}")

## $c_{0,unp}^{DVCS}$

In [75]:
def calculate_fourier_coefficients_unpolarized_DVCS_c0(
        k_shorthand_squared: float,
        epsilon: float,
        squared_Q_momentum_transfer: float,
        x_Bjorken: float,
        squared_hadronic_momentum_transfer_t: float,
        lepton_energy_fraction_y: float,
        compton_form_factor_h: float,
        compton_form_factor_h_tilde: float,
        compton_form_factor_e: float,
        compton_form_factor_e_tilde: float,
        verbose = True) -> float:
    """
    Description
    --------------
    Equation (43) of the BH02 Formalism.

    Parameters
    --------------
    k_shorthand_squared: (float)

    epsilon: (float)

    squared_Q_momentum_transfer: (float)

    x_Bjorken: (float)

    squared_hadronic_momentum_transfer_t: (float)

    lepton_energy_fraction_y: (float)

    compton_form_factor_h: (float)

    compton_form_factor_h_tilde: (float)

    compton_form_factor_e: (float)

    compton_form_factor_e_tilde: (float)

    verbose: (bool)
        Debugging console output.

    Notes
    --------------
    """
    FORMALISM = "BKM10"

    if verbose:
        print(f"> [{__name__}]: Now beginning calculation of c_0_unpolarized.")

    try:

        # (1): Calculate the prefactor of the coefficient: BKM02
        if FORMALISM == "BKM02":
            coefficient_first_prefactor = 2.* ( 2. - 2. * lepton_energy_fraction_y + lepton_energy_fraction_y**2)

        elif FORMALISM == "BKM10":
            coefficient_first_prefactor = 2.* ( 2. - 2. * lepton_energy_fraction_y + lepton_energy_fraction_y**2 + (lepton_energy_fraction_y**2 * epsilon**2 / 2.)) / (1. + epsilon**2)

        else:
            coefficient_first_prefactor = 2.* ( 2. - 2. * lepton_energy_fraction_y + lepton_energy_fraction_y**2 + (lepton_energy_fraction_y**2 * epsilon**2 / 2.)) / (1. + epsilon**2)
    
        # (2): Calculate the prefactor that appears in BKM10:
        if FORMALISM == "BKM02":
            coefficient_second_prefactor = 0
        elif FORMALISM == "BKM10":
            coefficient_second_prefactor = 16. * k_shorthand_squared / ((2. - x_Bjorken)**2 * (1. + epsilon**2))

        # (2): Calculate the insane coefficient with the Form Factors:
        dvcs_unpolarized_coefficient_with_form_factors = calculate_C_function_of_CFFs_unpolarized(
            epsilon,
            squared_Q_momentum_transfer,
            x_Bjorken,
            squared_hadronic_momentum_transfer_t,
            compton_form_factor_h,
            compton_form_factor_h_tilde,
            compton_form_factor_e,
            compton_form_factor_e_tilde
        )

        # (2): Calculate the insane coefficient with the Effective Form Factors:
        dvcs_unpolarized_coefficient_with__effective_form_factors = calculate_C_function_of_CFFs_unpolarized(
            epsilon,
            squared_Q_momentum_transfer,
            x_Bjorken,
            squared_hadronic_momentum_transfer_t,
            compton_form_factor_h,
            compton_form_factor_h_tilde,
            compton_form_factor_e,
            compton_form_factor_e_tilde
        )

        # (3): Calculate the entire coefficient:
        c0_dvcs_unpolarized_coefficient = coefficient_first_prefactor * dvcs_unpolarized_coefficient_with_form_factors + coefficient_second_prefactor * dvcs_unpolarized_coefficient_with__effective_form_factors
        
        if verbose:
            print(f"> Calculated c0_dvcs_unpolarized_coefficient to be: {c0_dvcs_unpolarized_coefficient}")

        return c0_dvcs_unpolarized_coefficient
    
    except Exception as ERROR:
        print(f"> Error in computing c0_dvcs_unpolarized:\n> {ERROR}")
        return None



In [76]:
def calculate_dvcs_amplitude_squared(
    lab_kinematics_k: float,
    squared_Q_momentum_transfer: float, 
    x_Bjorken: float,
    squared_hadronic_momentum_transfer_t: float,
    azimuthal_phi: float,
    compton_form_factor_h: float,
    compton_form_factor_h_tilde: float,
    compton_form_factor_hT: float,
    compton_form_factor_hT_tilde: float,
    compton_form_factor_e: float,
    compton_form_factor_e_tilde: float,
    compton_form_factor_eT: float,
    compton_form_factor_eT_tilde: float,
    verbose = True):
    """
    Description
    --------------
    Calculate the DVCS amplitude squared.

    Parameters
    --------------
    shorthand_k: (float)
    
    epsilon: (float)

    squared_Q_momentum_transfer: (float)

    x_Bjorken: (float)

    squared_hadronic_momentum_transfer_t: (float)

    lepton_energy_fraction_y: (float)

    azimuthal_phi: (float)

    compton_form_factor_h: (float)

    compton_form_factor_h_tilde: (float)

    compton_form_factor_hT: (float)

    compton_form_factor_hT_tilde: (float)

    compton_form_factor_e: (float)

    compton_form_factor_e_tilde: (float)

    compton_form_factor_eT: (float)

    compton_form_factor_eT_tilde: (float)

    verbose: (bool)
        Debugging console output.

    Notes
    --------------
    (1): The equation for the amplitude squared comes from
        Eq. (26) in this paper:
        https://arxiv.org/pdf/hep-ph/0112108.pdf
    """

    if verbose:
        verbose_input = True
    else:
        verbose_input = False

    try:

        # (1): Calculate the derived kinematic quantities:

        # (1.1): Calculate Epsilon:
        epsilon = calculate_kinematics_epsilon(
            np.sqrt(squared_Q_momentum_transfer),
            x_Bjorken,
            verbose_input
        )

        # (1.2): Calculate Epsilon Squared:
        epsilon_squared = calculate_kinematics_epsilon_squared(
            squared_Q_momentum_transfer,
            x_Bjorken,
            verbose_input
        )

        # (1.3): Calculate the Lepton Energy Fraction:
        lepton_energy_fraction_y = calculate_kinematics_lepton_energy_fraction_y(
            epsilon,
            squared_Q_momentum_transfer,
            lab_kinematics_k,
            verbose_input
        )

        # (1.4): Calculate the Skewness Parameter:
        skewness_parameter = calculate_kinematics_skewness_parameter(
            squared_Q_momentum_transfer,
            x_Bjorken,
            squared_hadronic_momentum_transfer_t,
            verbose_input
        )

        # (1.5): Calculate t_minimum
        squared_hadronic_momentum_transfer_t_minimum = calculate_kinematics_minimum_t(
            skewness_parameter,
            verbose_input
        )

        # (1.6): Calculate K^{2}:
        k_shorthand_squared = calculate_kinematics_k_squared(
            epsilon,
            squared_Q_momentum_transfer,
            x_Bjorken,
            lepton_energy_fraction_y,
            squared_hadronic_momentum_transfer_t,
            squared_hadronic_momentum_transfer_t_minimum,
            verbose_input
        )

        # (2): Calculate the prefactor of the amplitude (no electron charge -- cancels with overall prefactor)
        amplitude_prefactor = 1. / (lepton_energy_fraction_y**2 * squared_Q_momentum_transfer)

        # (3): Calculate the Fourier Modes:
        
        # (3.1): Obtain the first coefficient in the sum:
        coefficient_c0_DVCS = calculate_fourier_coefficients_unpolarized_DVCS_c0(
            k_shorthand_squared,
            epsilon,
            squared_Q_momentum_transfer,
            x_Bjorken,
            squared_hadronic_momentum_transfer_t, 
            lepton_energy_fraction_y,
            compton_form_factor_h,
            compton_form_factor_h_tilde,
            compton_form_factor_e,
            compton_form_factor_e_tilde,
            verbose_input
        )

        # # (3): Obtain the first coefficient in the unevaluated sum (cosine n = 1 term):
        # coefficient_c1_DVCS = calculate_fourier_coefficients_unpolarized_DVCS_c1(
        #     shorthand_k,
        #     x_Bjorken, 
        #     squared_hadronic_momentum_transfer_t, 
        #     lepton_energy_fraction_y,
        #     compton_form_factor_h,
        #     compton_form_factor_h_tilde,
        #     compton_form_factor_e,
        #     compton_form_factor_e_tilde,
        # )

        # # (4): Obtain the second coefficient in the unevaluated sum (cosine n = 2 term):
        # coefficient_c2_DVCS = calculate_fourier_coefficients_unpolarized_DVCS_c2(
        #     shorthand_k,
        #     squared_Q_momentum_transfer,
        #     x_Bjorken,  
        #     squared_hadronic_momentum_transfer_t,
        #     compton_form_factor_h,
        #     compton_form_factor_h_tilde,
        #     compton_form_factor_hT,
        #     compton_form_factor_hT_tilde,
        #     compton_form_factor_e,
        #     compton_form_factor_e_tilde,
        #     compton_form_factor_eT,
        #     compton_form_factor_eT_tilde,
        # )

        # # (5): Obtain the first coefficient in the unevaluated sum (sine n = 1 term):
        # coefficient_s1_DVCS = calculate_fourier_coefficients_unpolarized_DVCS_s1(
        #     x_Bjorken, 
        #     squared_hadronic_momentum_transfer_t, 
        #     lepton_energy_fraction_y,
        #     compton_form_factor_h,
        #     compton_form_factor_h_tilde,
        #     compton_form_factor_e,
        #     compton_form_factor_e_tilde,
        # )

        # (6): Compute the Fourier Mode Expansion:
        # mode_expansion = coefficient_c0_DVCS + (coefficient_c1_DVCS * np.cos(azimuthal_phi)) + (coefficient_c2_DVCS * np.cos(2. * azimuthal_phi)) + (coefficient_s1_DVCS * np.sin(azimuthal_phi)) 

        mode_expansion = coefficient_c0_DVCS
        
        # (7): The entire amplitude:
        dvcs_amplitude_squared = amplitude_prefactor * mode_expansion

        if verbose:
            print(f"> Calculated DVCS amplitude squared as: {dvcs_amplitude_squared}")

        # (8): Return the amplitude:
        return dvcs_amplitude_squared
    
    except Exception as E:
        print(f"> Error in calculating the DVCS amplitude squared\n> {E}")
        return None

In [77]:
def read_csv_file_with_pandas(name_of_csv_file: str) -> pd.DataFrame:
    """
    Description
    --------------
    We are using Pandas to read the .csv file. If this works, then the
    function will return the 

    Parameters
    --------------
    name_of_csv_file: str
    
    Returns
    --------------
    pandas_read_csv: Pandas DF

    Function Flow
    --------------
    (1): Try to read the CSV with Pandas. If it works,
        it works. YAY! If it don't, then return None.
    
    Notes
    --------------
    """
    try:
        pandas_read_csv = pd.read_csv(name_of_csv_file)
        return pandas_read_csv
    except Exception as E:
        print(f"> Error reading the .csv file with Pandas:\n> {E}")
        return None

In [78]:
jlab_pandas_df = read_csv_file_with_pandas('jlab_kinematics.csv')
jlab_pandas_df.head()

,k,QQ,x_b,t,phi_x,F,sigmaF,ReH,ReE,ReHt,dvcs
0,5.75,1.82,0.343,-0.172,7.5,0.120053,0.00492,-0.992404,-0.31,-0.396272,0.017761
1,5.75,1.82,0.343,-0.172,22.5,0.114969,0.00468,-0.992404,-0.31,-0.396272,0.017761
2,5.75,1.82,0.343,-0.172,37.5,0.106078,0.00420,-0.992404,-0.31,-0.396272,0.017761
3,5.75,1.82,0.343,-0.172,52.5,0.095266,0.00396,-0.992404,-0.31,-0.396272,0.017761
4,5.75,1.82,0.343,-0.172,67.5,0.084249,0.00360,-0.992404,-0.31,-0.396272,0.017761


In [79]:
calculate_dvcs_amplitude_squared(
    jlab_pandas_df['k'][1],
    jlab_pandas_df['QQ'][1],
    jlab_pandas_df['x_b'][1],
    jlab_pandas_df['t'][1],
    jlab_pandas_df['phi_x'][1],
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    False
)

> [__main__]: Now beginning calculation of C_function_unp.
> [__main__]: Calculated `a_factor` to be: -0.04884396104354504
> [__main__]: Calculated `b_factor` to be: 0.6570000052452087
> [__main__]: Calculated `c_factor` to be: 1.761004051350713
> [__main__]: Calculated `d_factor` to be: -0.3028927071187967
> [__main__]: Calculated `e_factor` to be: 2.956744095358015
> [__main__]: Calculated `f_factor` to be: 1.6480000466108324
> [__main__]: Calculated `prefactor` to be: -0.034646657301127134
> [__main__]: Calculated `cff_h_h_tilde_contribution_first_term` to be: 2.628000020980835
> [__main__]: Calculated `cff_h_h_tilde_contribution_second_term` to be: 3.719324262121285
> [__main__]: Calculated `cff_e_h_mixing_contribution` to be: 6.151056717810013
> [__main__]: Calculated `cff_e_tilde_h_tilde_mixing_contribution` to be: -0.7089819105935279
> [__main__]: Calculated `cff_e_estar_contribution` to be: 1.6657542320960679
> [__main__]: Calculated `cff_e_tilde_e_tilde_star_contribution` to b

-3.020182940597524